In [36]:
import numpy as np
from collections import Counter
from numpy import genfromtxt
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = genfromtxt('kc2.csv', delimiter = ',')
np.random.seed(15)
np.random.shuffle(df)

In [5]:
class KNN:
  def __init__(self, k=1, dist='euclidean'):
    self.k = k
    self.dist = dist

  def euclidean_distance(self, x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

  def mahalanobis_distance(self, x1, x2):
    return np.sqrt(np.dot(np.dot((x1-x2).T, np.linalg.pinv(np.cov(self.X, rowvar=False))),(x1-x2)))

  """
  Aqui em Mahalanobis, para algumas muitas seeds geravam conjuntos que formavam uma matriz de cov linear, de modo que sempre dava erro para calcular a inversa.
  Procurei bastante na net, e vi mais de 1 resposta recomendando usar a pseudo-inversa (Moore-Penrose) e acabei usando aqui... Pelo menos funcionou! Não sei se pode
  professor, imagino que o correto mesmo seria apenas a inversa. 
  """

  def fit(self, X, y):
    self.X_train = X
    self.y_train = y
  
  def predict(self, X):
    self.X = X
    predictions = [self._predict(x) for x in X]
    return predictions

  def _predict(self, x):
    if self.dist == 'euclidean':
      distances = [self.euclidean_distance(x, x_train) for x_train in self.X_train]

    if self.dist == 'mahalanobis':
      distances = [self.mahalanobis_distance(x, x_train) for x_train in self.X_train]

    k_ind = np.argsort(distances)[:self.k]
    kn_labels = [self.y_train[_] for _ in k_ind]
    return Counter(kn_labels).most_common()[0][0]

In [34]:
def get_metrics(ground_truth, prediction, metric):

  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for y, y_hat in zip(ground_truth, prediction):
    if y == 1 and y_hat == 1:
      tp += 1
    if y == 0 and y_hat == 0:
      tn += 1
    if y == 0 and y_hat == 1:
      fp += 1
    if y == 1 and y_hat == 0:
      fn += 1

  if metric == 'acc':
    return (tp + tn)/(tp + tn + fp + fn) 

  if metric == 'prec':
    try:
      return  tp/(tp + fp)  
    except ZeroDivisionError:
      return 0

  if metric == 'rec':
    try:
      return  tp/(tp + fn)  
    except ZeroDivisionError:
      return 0

  if metric == 'f1':
    return 2*tp/(2*tp + fp + fn)

In [35]:
k_folds = 10
n_samples = df.shape[0]
pointer_0 = 0
pointer_1 = n_samples // k_folds
pointer_final = n_samples % k_folds

train = []
test = []
y_test = []

for i in range(k_folds):
  if pointer_final > 0:
    pointer_final = pointer_final - 1
    te = df[pointer_0 : pointer_0 + pointer_1 + 1]
    tr = np.r_[df[0 : pointer_0], df[pointer_0 + pointer_1 + 1 : ]]
    pointer_0 = pointer_0 + pointer_1 + 1
  else:
    te = df[pointer_0 : pointer_0 + pointer_1]
    tr = np.r_[df[0 : pointer_0], df[pointer_0 + pointer_1 : ]]
    pointer_0 = pointer_0 + pointer_1 + 1
  train.append(tr)
  test.append(te)

KNN1_eu = []
KNN1_ma = []
KNN5_eu = []
KNN5_ma = []
DTC_gi = []
DTC_en = []

knn_1_eu = KNN(1, 'euclidean')
knn_1_ma = KNN(1, 'mahalanobis')
knn_5_eu = KNN(5, 'euclidean')
knn_5_ma = KNN(5, 'mahalanobis')
dtc_gi = DecisionTreeClassifier(criterion='gini')
dtc_en = DecisionTreeClassifier(criterion='entropy')

for i in range(k_folds):
  tr = train[i]
  te = test[i]
  X_train = tr[ : , : -1]
  X_test = te[ : , : -1]
  y_train = tr[ : , -1]
  y_test.append(te[ : , -1])

  knn_1_eu.fit(X_train, y_train)
  knn_1_ma.fit(X_train, y_train)
  knn_5_eu.fit(X_train, y_train)
  knn_5_ma.fit(X_train, y_train)
  dtc_gi.fit(X_train, y_train)
  dtc_en.fit(X_train, y_train)

  KNN1_eu.append(knn_1_eu.predict(X_test))
  KNN1_ma.append(knn_1_ma.predict(X_test))
  KNN5_eu.append(knn_5_eu.predict(X_test))
  KNN5_ma.append(knn_5_ma.predict(X_test))
  DTC_gi.append(dtc_gi.predict(X_test))
  DTC_en.append(dtc_en.predict(X_test))

for i in range(k_folds):
  print("=-"*20)
  print("-Fold: ", i+1)
  print("\n")
  print("---1NN (Euclidean Distance)---")
  print("Acurácia:", get_metrics(KNN1_eu[i], y_test[i], 'acc'))
  print("Precisão:", get_metrics(KNN1_eu[i], y_test[i], 'prec'))
  print("Revocação:", get_metrics(KNN1_eu[i], y_test[i], 'rec'))
  print("F1:", get_metrics(KNN1_eu[i], y_test[i], 'f1'))
  print("\n")
  print("---5NN (Euclidean Distance)---")
  print("Acurácia:", get_metrics(KNN5_eu[i], y_test[i], 'acc'))
  print("Precisão:", get_metrics(KNN5_eu[i], y_test[i], 'prec'))
  print("Revocação:", get_metrics(KNN5_eu[i], y_test[i], 'rec'))
  print("F1:", get_metrics(KNN5_eu[i], y_test[i], 'f1'))
  print("\n")
  print("---1NN (Mahalanobis Distance)---")
  print("Acurácia:", get_metrics(KNN1_ma[i], y_test[i], 'acc'))
  print("Precisão:", get_metrics(KNN1_ma[i], y_test[i], 'prec'))
  print("Revocação:", get_metrics(KNN1_ma[i], y_test[i], 'rec'))
  print("F1:", get_metrics(KNN1_ma[i], y_test[i], 'f1'))
  print("\n")
  print("---5NN (Mahalanobis Distance)---")
  print("Acurácia:", get_metrics(KNN5_ma[i], y_test[i], 'acc'))
  print("Precisão:", get_metrics(KNN5_ma[i], y_test[i], 'prec'))
  print("Revocação:", get_metrics(KNN5_ma[i], y_test[i], 'rec'))
  print("F1:", get_metrics(KNN5_ma[i], y_test[i], 'f1'))
  print("\n")
  print("---Decision Tree (Gini Index)---")
  print("Acurácia:", get_metrics(DTC_gi[i], y_test[i], 'acc'))
  print("Precisão:", get_metrics(DTC_gi[i], y_test[i], 'prec'))
  print("Revocação:", get_metrics(DTC_gi[i], y_test[i], 'rec'))
  print("F1:", get_metrics(DTC_gi[i], y_test[i], 'f1'))
  print("\n")
  print("---Decision Tree (Entropy Measure)---")
  print("Acurácia:", get_metrics(DTC_en[i], y_test[i], 'acc'))
  print("Precisão:", get_metrics(DTC_en[i], y_test[i], 'prec'))
  print("Revocação:", get_metrics(DTC_en[i], y_test[i], 'rec'))
  print("F1:", get_metrics(DTC_en[i], y_test[i], 'f1'))
  print("\n")

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
-Fold:  1


---1NN (Euclidean Distance)---
Acurácia: 0.7272727272727273
Precisão: 0.75
Revocação: 0.75
F1: 0.75


---5NN (Euclidean Distance)---
Acurácia: 0.7272727272727273
Precisão: 0.75
Revocação: 0.75
F1: 0.75


---1NN (Mahalanobis Distance)---
Acurácia: 0.4090909090909091
Precisão: 0.08333333333333333
Revocação: 0.3333333333333333
F1: 0.13333333333333333


---5NN (Mahalanobis Distance)---
Acurácia: 0.45454545454545453
Precisão: 0.0
Revocação: 0
F1: 0.0


---Decision Tree (Gini Index)---
Acurácia: 0.6363636363636364
Precisão: 0.5833333333333334
Revocação: 0.7
F1: 0.6363636363636364


---Decision Tree (Entropy Measure)---
Acurácia: 0.5909090909090909
Precisão: 0.5833333333333334
Revocação: 0.6363636363636364
F1: 0.6086956521739131


=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
-Fold:  2


---1NN (Euclidean Distance)---
Acurácia: 0.6818181818181818
Precisão: 0.7
Revocação: 0.6363636363636364
F1: 0.6666666666666666


---5NN (Euclidean Distance)---
